In [23]:
# Needed for functions
import numpy as np
import time
import math

# Import QISKit classes
import qiskit
from qiskit import QuantumRegister, QuantumCircuit, Aer
from qiskit.quantum_info import state_fidelity, process_fidelity
from qiskit.tools.qi.qi import outer

# Tomography functions
from qiskit.ignis.verification.tomography import process_tomography_circuits, ProcessTomographyFitter

In [24]:
#Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])
circ.x(q[0])
circ.ry(np.pi/2, q)

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=100)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)
qpt_tomo.data

{(('Zp',), ('X',)): {'0': 56, '1': 44},
 (('Zp',), ('Y',)): {'0': 54, '1': 46},
 (('Zp',), ('Z',)): {'1': 100},
 (('Zm',), ('X',)): {'0': 48, '1': 52},
 (('Zm',), ('Y',)): {'0': 46, '1': 54},
 (('Zm',), ('Z',)): {'0': 100},
 (('Xp',), ('X',)): {'1': 100},
 (('Xp',), ('Y',)): {'0': 45, '1': 55},
 (('Xp',), ('Z',)): {'0': 47, '1': 53},
 (('Yp',), ('X',)): {'0': 47, '1': 53},
 (('Yp',), ('Y',)): {'0': 100},
 (('Yp',), ('Z',)): {'0': 47, '1': 53}}

In [25]:
# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_lstsq = qpt_tomo.fit(method='lstsq')
print('Least-Sq Fitter')
print('fit time:', time.time() - t)
print('fit fidelity (state):', state_fidelity(choi_ideal / 2, choi_lstsq.data / 2))
print('fit fidelity (process):', np.real(process_fidelity(choi_ideal, choi_lstsq.data, require_cptp=False)))


# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = qpt_tomo.fit(method='cvx')
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity (state):', state_fidelity(choi_ideal / 2, choi_cvx.data / 2))
print('fit fidelity (process):', np.real(process_fidelity(choi_ideal, choi_cvx.data, require_cptp=False)))



Least-Sq Fitter
fit time: 0.010769128799438477
fit fidelity (state): 0.9911504456111692
fit fidelity (process): 0.9823792050511149

CVXOPT Fitter
fit time: 0.04892325401306152
fit fidelity (state): 0.9974400353274551
fit fidelity (process): 0.9948866240728369


In [26]:
# Process tomography of a Hadamard gate
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.swap(q[0], q[1])

# Ideal channel is a unitary
ideal_unitary = np.eye(2)
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
# We use the optional prepared_qubits kwarg to specify that the prepared qubit was different to measured qubit
qpt_circs = process_tomography_circuits(circ, q[1], prepared_qubits=q[0])
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)
qpt_tomo.data

{(('Zp',), ('X',)): {'0': 995, '1': 1005},
 (('Zp',), ('Y',)): {'0': 996, '1': 1004},
 (('Zp',), ('Z',)): {'0': 2000},
 (('Zm',), ('X',)): {'0': 988, '1': 1012},
 (('Zm',), ('Y',)): {'0': 1008, '1': 992},
 (('Zm',), ('Z',)): {'1': 2000},
 (('Xp',), ('X',)): {'0': 2000},
 (('Xp',), ('Y',)): {'0': 965, '1': 1035},
 (('Xp',), ('Z',)): {'0': 958, '1': 1042},
 (('Yp',), ('X',)): {'0': 1022, '1': 978},
 (('Yp',), ('Y',)): {'0': 2000},
 (('Yp',), ('Z',)): {'0': 986, '1': 1014}}